**Note:** This notebook is written in the Julia language, so the cells
  can't be executed in Google Colab. If you want to verify that the
  notebook works, I recommend [JuliaBox](https://juliabox.com/) (which
  is free and requires no installation) or testing locally. The syntax
  is very similar to Python and MATLAB. Note in particular the dot
  syntax used to perform elementwise operations ( `f.(x)` applies `f`
  to all elements of `x` ), that indices start at 1 and that the last
  statement of all functions is returned automatically.

**Extra note:** The creators of Julia recently received the Wilkinson
  Prize for Numerical Software, a prize also awarded to the creator of
  the Triangle mesh generator. Here is a
  [talk](https://www.youtube.com/watch?v=OfMP5PTFQk0&feature=youtu.be)
  from the event.

# **Lab 7: Optimization and learning**
**Anders Ågren Thuné**

# **Abstract**

# **About the code**

In [1]:
"""
DD2363 Methods in Scientific Computing,
KTH Royal Institute of Technology, Stockholm, Sweden.
"""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

"DD2363 Methods in Scientific Computing,\nKTH Royal Institute of Technology, Stockholm, Sweden.\n"

# **Set up environment**

In [2]:
using LinearAlgebra

# **Introduction**

Optimization is the study of finding the optimal value of some domain,
typically the maximum or minimum of some function. This is a field
with a broad range of applications, not least in the rapidly growing
field of machine learning.

In order to find the critical points of a function, the derivative
or multivariate versions thereof are typically powerful tools, as they
indicate the direction to move in the input space to achieve the largest
change in the output, the 'steepest slope' in a sense.

Two methods making use of this fact to perform unconstrained
optimization in $\mathbb{R}^n$ are the gradient descent method and the
Newton method. The gradient descent method is typically used when
training artificial neural networks, as it is a relatively cheap way
to find minima of a cost function, especially when a stochastic
version is used.

[Hoffman 2019, Chapter 19]

This report presents how the gradient descent and Newton methods were
implemented in the Julia language. In addition, a simple feedforward
neural network was implemented and trained using the backpropagation
algorith.

# **Methods**

### **Gradient descent and Newton's method**

Gradient descent and Newton's method are very similar in that they
utilize the fact that the gradient indicates the direction of steepest
descent to step through the input space to search for a
minimum. However, while gradient descent uses a step size $\alpha^{k}$
determined somewhat heuristically, Newton's method uses the Hessian,
resulting in a somewhat more expensive method, but with a higher order
of convergence.

Gradient descent: $$x^{(k+1)} = x^{(k)} - \alpha^{(k)}\nabla
f(x^{(k)}),$$ $$\text{for $\alpha^{(k)}$ so that}\ f(x^{(k+1)}) \leq
\beta f(x^{(k)}), \quad \text{where}\ \beta < 1\ \text{is a fixed
parameter}$$

Newton's method: $$x^{(k+1)} = x^{(k)} - (Hf(x^{(k)}))^{-1}\nabla
f(x^{(k)}) $$

In [ ]:
"""
Calculate J(f)(p), modifying `jac` in place. Reused from lab 5.
"""
function filljacobian!(jac, f, p, h = 1e-5)
    p1 = p
    p2 = copy(p1)
    for i in 1:length(p)
        p1[i] += h
        p2[i] -= h

        jac[:,i] .= (f(p1) - f(p2)) / 2h

        p1[i] -= h
        p2[i] += h
    end
end

"""
Calculate ∇(f)(x), modifying v in place.
"""
fillgradient!(v, f, p, h = 1e-5) = filljacobian!(v', f, p, h)

"""
Use gradient descent to find x* such that ||∇(f)(x*)|| < TOL
"""
function gradient_descent(f, β, x⁽⁰⁾, TOL = 1e-6)
    x = x⁽⁰⁾
    ∇f = Inf*ones(length(x⁽⁰⁾))
    while norm(∇f) >= TOL
        fillgradient!(∇f, f, x)
        α = find_α(f, β, x, ∇f)
        x .-= α .* ∇f
    end
end

"""
Use Newton's method to find x* such that ||∇(f)(x*)|| < TOL
"""
function newton_optimization(f, H, x⁽⁰⁾, TOL = 1e-6)
    x = x⁽⁰⁾
    ∇f = Inf*ones(length(x⁽⁰⁾))
    while norm(∇f) >= TOL
        fillgradient!(∇f, f, x)
        x .-= H(x) \ ∇f
    end
end

# **Results**

# **Discussion**

# **References**

- Hoffman, J. 2019. *Introduction to Scientific Computing*